In [ ]:
#!pip install langchain==0.2.14
#!pip install langchain_community==0.2.14
#!pip install pydantic==1.10.8
#!pip install gradio==4.44.1
!pip install -U openai langchain langchain-openai gradio ipywidgets

In [ ]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.pdf', multiple=True)
display(uploader)

In [ ]:
print(uploader.value[0])

In [ ]:
bytes_data = uploader.value[0].content.tobytes()

In [ ]:
print(type(bytes_data))

In [ ]:
import os

def save_uploaded_pdf(uploader):
    uploaded_file = list(uploader.value)[0]
    file_name = uploaded_file['name']
    save_path = os.path.join(os.getcwd(), file_name)
    with open(save_path, 'wb') as f:
        f.write(uploaded_file['content'])
    
    print(f'Uploaded and saved: {file_name}')
    return file_name

file_name = save_uploaded_pdf(uploader)

In [ ]:
!pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_name)
pages = loader.load_and_split()

In [ ]:
pages

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_name)
documents = loader.load()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

In [ ]:
docs[:3]

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.9)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
#embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("shower")

In [ ]:
index.query("소년은 어디에서 처음 소녀를 만났나?", llm=chat, verbose=True)

In [ ]:
index.query("소년은 소녀에게 무엇을 주려고 했나?", llm=chat, verbose=True)

In [ ]:
index.query("소녀는 소년에게 무엇을 주었나?", llm=chat, verbose=True)

In [ ]:
index.query("소녀와 윤초시는 어떤 관계인가?", llm=chat, verbose=True)